In [1]:
import torch
import torch.nn as nn
from autograd import grad, jacobian
from math import pi #to get the definition of pi
import pandas as pd

data = pd.read_csv('Laplacian.csv', header=None) #let's import the grid
X = torch.tensor(data.values[:, 0:2], requires_grad=True).to(torch.float32)

We are still working on the laplacian equation $-\Delta u(x,y)= f(x,y)$.

According to this paper https://arxiv.org/pdf/physics/9705023.pdf, we have to write the output of our neural network as :

$$ \Psi_t(x,y) = A(x,y) + x(1-x)y(1-y)N(x,y,p)$$

were A is the part that takes care of the boundary conditions and p is the weight matrix of the neural net.

In the end, the loss function is $ E[p] = \sum_i \{\frac{\partial}{\partial x^2}\Psi(x_i,y_i) + \frac{\partial}{\partial y^2}\Psi(x_i,y_i) - f(x_i, y_i)\}^2$


First let's deal with the boundary conditions and the f function

In [2]:
alpha = pi/2 ; beta = pi # We use those to be able to compare to an exact solution
def A(X) : return torch.sin(alpha*X[:,0])*torch.cos(beta*X[:,1])
def f(X) : return (alpha**2 + beta**2)*torch.sin(alpha*X[:,0])*torch.cos(beta*X[:,0])

Then we can write the Psi function

In [3]:
def Psi(X, N) : return A(X) + X[:,0]*(1-X[:,0])*X[:,1]*(1-X[:,1]) * N.t()

Let's play around to see if torch.autograd.grad is giving us what we need

In [4]:
W = torch.randn(2,1, requires_grad=True)
N = X@W; 
output = Psi(X, N).sum()
jac = torch.autograd.grad(output, X, create_graph=True); #jac

In [5]:
# print (torch.autograd.grad(jac[0][:,0].sum(),X, create_graph=True)[0][:,1] -
#torch.autograd.grad(jac[0][:,1].sum(),X, create_graph=True)[0][:,0]) 
#just to check if cross derivative are equal

Okay, now we have an idea about how to compute Jacobian and hessian matrices, let's build our loss function.

In [6]:
def Loss(X,N) :
    N = model(X)
    output = Psi(X, N).sum()
    jac = torch.autograd.grad(output, X, create_graph=True)
    hes_x = torch.autograd.grad(jac[0][:,0].sum(),X, create_graph=True)[0][:,0]
    hes_y = torch.autograd.grad(jac[0][:,1].sum(),X, create_graph=True)[0][:,1]
    
    loss =(((hes_x + hes_y) - f(X))**2).mean()
    return loss

We can now build our neural network

In [33]:
class SimpleNet(nn.Module):
    # Initialize the layers
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(2, 64, bias=True) #multipliyin the input by some weigth
        self.linear2 = nn.Linear(64, 1)
        
    def forward(self,x):
        x = torch.sigmoid(self.linear1(x))
        x = self.linear2(x)
        return x

In [34]:
model = SimpleNet() #the network we created

In [41]:
def fit(epochs, lr=0.001):
    opt = torch.optim.Adam(model.parameters(), lr, weight_decay=0.01) #gradient descent
    
    for epoch in range(epochs):
        epoch +=1
        opt.zero_grad()
        loss = Loss(X,N)
        loss.backward()# back props
        opt.step()# update the parameters
        if epoch % (epochs//10) == 0: print('epoch {}, loss {}'.format(epoch, loss.data))

In [42]:
fit(1000,0.1)

epoch 100, loss 10.731881141662598
epoch 200, loss 8.073054313659668
epoch 300, loss 6.1805291175842285
epoch 400, loss 5.4158854484558105
epoch 500, loss 5.155508518218994
epoch 600, loss 5.121185779571533
epoch 700, loss 5.086486339569092
epoch 800, loss 5.168694019317627
epoch 900, loss 5.051408290863037
epoch 1000, loss 5.059935569763184
